In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
!pip install segmentation_models
!pip install albumentations==0.4.5

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
import tensorflow as tf

from IPython.display import clear_output
import matplotlib.pyplot as plt

In [6]:
from PIL import Image

import os
import glob
import numpy as np

from sklearn import model_selection

import segmentation_models as sm

Segmentation Models: using `tf.keras` framework.


In [7]:
from src.DataLoader import DataLoader, make_augmentation
from src.Model import unet_model

In [8]:
base_dir = "/storage/supervisely/processed"
ims_dir = os.path.join(base_dir, "imgs")
labels_dir = os.path.join(base_dir, "labels")
img_files = sorted(glob.glob(ims_dir + "/*.*"))
mask_files = sorted(glob.glob(labels_dir + "/*.png"))
print (f"Masks count : {len(mask_files)};\nImgs count : {len(img_files)};")

Masks count : 5709;
Imgs count : 5709;


In [9]:
BATCH_SIZE = 24
dataset = list(zip(img_files, mask_files))
train_dataset, test_dataset = model_selection.train_test_split(dataset, test_size=0.2, random_state=0)
print(f"Train dataset size {len(train_dataset)}; Test dataset size {len(test_dataset)}")

Train dataset size 4567; Test dataset size 1142


In [17]:
output_size = (256, 256)

validation_augmentation = make_augmentation(output_size=output_size, is_validation=False)
train_data_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, augmentation_fn=validation_augmentation, output_size=output_size, shuffle=False)

validation_augmentation = make_augmentation(output_size=output_size, is_validation=True)
test_data_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, augmentation_fn=validation_augmentation, output_size=output_size, shuffle=False)

## Measure batch access

In [15]:
%%timeit 
batch = test_data_loader[0]

7.96 s ± 2.15 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
OUTPUT_CHANNELS = 1
model = unet_model(OUTPUT_CHANNELS)

dice_loss = sm.losses.DiceLoss()
total_loss = dice_loss
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]
# compile keras model with defined optimozer, loss and metrics
model.compile(optim, total_loss, metrics)

history = model.fit(
    x = train_data_loader, 
    steps_per_epoch=10, 
    epochs=2, 
    callbacks=callbacks, 
    use_multiprocessing=False,
    workers=1
) 

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


KeyboardInterrupt: 